In [2]:
!pip3 install python-Levenshtein
%pip install jiwer
%pip install nest-asyncio
!pip3 install pydub
import nest_asyncio
nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Ensure this notebook is in the NBL_GroundTruth_DataSet folder

In [3]:
import pandas as pd

audio_segments_path_0 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-000'
audio_segments_path_1 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-001'
audio_segments_path_2 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-002'
audio_segments_path_3 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-003'

csv_0 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-000.csv'
csv_1 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-001.csv'
csv_2 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-002.csv'
csv_3 = '/Users/sathv/Desktop/NBL_GroundTruth_DataSet/EV1-MAURER-2021-04-02_08-37-06-003.csv'

df_0 = pd.read_csv(csv_0)
df_1 = pd.read_csv(csv_1)
df_2 = pd.read_csv(csv_2)
df_3 = pd.read_csv(csv_3)

Optionally, mount your Google Drive with your Colab session. Note that you will need to add a shortcut to this folder to your homepage and add the path to each string in reading the .csv's and audio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
#create dict containing wav as key and text as value
def create_times_dict(df : pd.DataFrame):
    times = {}
    for row, col in df.iterrows():
        try:
            filename = df.iloc[row][1].replace(':', '_')
            #print(filename)
            times[filename] = df.iloc[row][2]
        except IndexError:
            print('')
    return times
times_0 = create_times_dict(df_0)
print(times_0)

{'0_1_5-0_1_8.wav': 'Copy suit engineer. Thank you.', '0_2_45-0_2_51.wav': 'Attention in the MBO, we are about to begin test all duty station personnel. Please report to your assignments.', '0_2_59-0_3_4.wav': 'CVO verify video is being recorded on server and allow audio to leave the NBL.', '0_3_5-0_3_11.wav': 'TDC video is being recorded on server. Audio is not leaving the NBL. Copy.', '0_3_26-0_3_29.wav': 'Did you agree?', '0_3_29-0_3_31.wav': 'Are you a go?', '0_3_47-0_3_52.wav': "Copy. TC. TC's go? TSO.", '0_3_52-0_4_7.wav': "M e t. M e t's go. Suit engineer. Suit engineer's go. Suit tech's one and two. One go. Cheers go. SSRMS, DCR, and RCCW. SSR, MS, DCR, and RCCW, you go. Copy all.", '0_4_13-0_4_22.wav': 'Copy. Topside, verify donning areas, clear ignition sources, and hand sanitizer. TD topside, donning area is clear ignition sources, and hand sanitizer.', '0_4_32-0_4_38.wav': 'TCT taking one, if this last is again secure, the bill connector installed and locked.', '0_4_43-0_4_

In [ ]:
times_1 = create_times_dict(df_1)
print(times_1)

In [ ]:
times_2 = create_times_dict(df_2)
print(times_2)

In [ ]:
times_3 = create_times_dict(df_3)
print(times_3)

In [6]:
# Code for similarity metric
from difflib import SequenceMatcher
from jiwer import wer, mer, wil
from Levenshtein import distance

def metrics(asr, ground_truth : str, transcr : str):
    m_wer = round(wer(ground_truth, transcr),2)
    m_mer = round(mer(ground_truth, transcr),2)
    m_wil = round(wil(ground_truth, transcr),2)
    m_sim = round(SequenceMatcher(None, ground_truth, transcr).ratio(),2)
    m_dist = round(distance(transcr, ground_truth),2)
    print(asr, '-','WER:',m_wer, 'MER:',m_mer,'WIL:',m_wil,'SIM:',m_sim,'L-DIST:',m_dist)
    return m_wer, m_mer, m_wil, m_sim, m_dist

def avg(metrics):
  return str(round(sum(metrics) / len(metrics), 2))

In [10]:
#You can either write a function that accepts a .wav audio file path and transribes it to return text
from google.cloud import speech
import argparse

def model_transcribe_function(wav_file: str) -> speech.RecognizeResponse:
   
    client = speech.SpeechClient()
    with open(wav_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(f"Transcript: {result.alternatives[0].transcript}")
        #print(f"Confidence: {result.alternatives[0].confidence}")
        print(f"Full Result: {result}")

    return result.alternatives[0].transcript
#or write a similar function that uses a list of previous transcriptions to compare them to ground truth transcriptions




In [11]:

#Test ASR

def evaluate_transcr(times : dict):
    #lists to contain metrics for both ASR scores
    print("hello")
    model_wer = []
    model_mer = []
    model_wil = []
    model_sim = []
    model_dist = []

    for key, value in times.items():
        #run transcription function
        model_transcription_text = model_transcribe_function('{}/{}'.format(audio_segments_path_0, key))
    
        #print results from both ASRs
        print('\n', key, 'Ground Truth:', value)
        print('Model Transcription:', model_transcription_text)

        #print ASR metrics
        wer, mer, wil, sim, dist = metrics('Model', value, model_transcription_text)

        #append metrics to ASR score lists

        model_wer.append(wer)
        model_mer.append(mer)
        model_wil.append(wil)
        model_sim.append(sim)
        model_dist.append(dist)

    #print overall ASR metrics
    print('Average ASR Metrics - WER:', avg(model_wer), 'MER:', avg(model_mer), 'WIL:', avg(model_wil), 'SIM:', avg(model_sim), 'L-DIST:', avg(model_dist))
    model_metrics = {'wer': float(avg(model_wer)), 'mer': float(avg(model_mer)), 'wil' : float(avg(model_wil)), 'sim' : float(avg(model_sim)), 'l_dist' : float(avg(model_dist))}
    return model_metrics


In [12]:
evaluate_transcr(times_0)


hello
Transcript: copy city engineer thank you
Full Result: alternatives {
  transcript: "copy city engineer thank you"
  confidence: 0.813854039
}
result_end_time {
  seconds: 2
  nanos: 320000000
}
language_code: "en-us"


 0_1_5-0_1_8.wav Ground Truth: Copy suit engineer. Thank you.
Model Transcription: copy city engineer thank you
Model - WER: 1.0 MER: 1.0 WIL: 1.0 SIM: 0.83 L-DIST: 7
Transcript: attention in the NBI we are about to begin test on duty station Personnel please report to your assignments
Full Result: alternatives {
  transcript: "attention in the NBI we are about to begin test on duty station Personnel please report to your assignments"
  confidence: 0.923928916
}
result_end_time {
  seconds: 5
  nanos: 810000000
}
language_code: "en-us"


 0_2_45-0_2_51.wav Ground Truth: Attention in the MBO, we are about to begin test all duty station personnel. Please report to your assignments.
Model Transcription: attention in the NBI we are about to begin test on duty station P

UnboundLocalError: local variable 'result' referenced before assignment